# 

In [1]:
!pip install arsenic
!pip install requests_html

     |████████████████████████████████| 49 kB 10.7 MB/s 
     |████████████████████████████████| 1.3 MB 33.2 MB/s 
     |████████████████████████████████| 294 kB 83.7 MB/s 
     |████████████████████████████████| 142 kB 92.0 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 115 kB 89.6 MB/s 
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=7908cb7e122c29dbd663475f1cdaf2d336dd9c036afee3d9d0a84e996eac936f
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13486 sha256=9d7c9d20d8807dbbcc69deb770610f3c80b421361724c6428a9f3610ec5001d4
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=245

In [2]:
!pip install chromedriver

     |████████████████████████████████| 14.3 MB 15.6 MB/s 


In [3]:
pip install structlog

Note: you may need to restart the kernel to use updated packages.


In [4]:
%%writefile async_scrape_multiple.py
import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML 
import itertools
import re
import time
import pathlib
import pickle
import pandas as pd

import logging
import structlog #pip install structlog


def store_links_as_df_pickle(datas=[], name='links2.pkl'):
    new_df = pd.DataFrame(datas)
    og_df = pd.DataFrame([{'id':0}])
    if pathlib.Path(name).exists():
        og_df = pd.read_pickle(name)
    df = pd.concat([og_df, new_df])
    df.reset_index(inplace=True, drop=False)
    df = df[['id', 'slug', 'path', 'scraped']]
    df = df.loc[-df.id.duplicated(keep='first')]
    df.dropna(inplace=True)
    df.to_pickle(name)
    return df

def set_arsenic_log_level(level = logging.WARNING):
    #Create logger
    logger = logging.getLogger('arsenic')

    def logger_factory():
        return logger

    structlog.configure(logger_factory=logger_factory)
    logger.setLevel(level)


async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']

async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]

    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            'id': id_,
            'slug': slug_,
            'path': path,
            'scraped': 0
        }
        datas.append(data)
    return datas

async def scraper(url, i=-1, timeout=60, start=None):
    service = services.Chromedriver()
    browser = browsers.Chrome()
    browser.capabilities = {
        "goog:chromeOptions": {"args": ["--headless", "--disable-gpu", "--no-sandbox", "--disable-dev-shm-usage"]}
    }
    async with get_session(service, browser) as session:
        try:
            await asyncio.wait_for(session.get(url), timeout=timeout)
        except asyncio.TimeoutError:
            return []
        body = await session.get_page_source()
        links = await get_links(body)
        if start != None:
            end = time.time() - start
            print(f'{i} took {end} seconds')
        # print(body)
        return links

async def run(urls, timeout=60, start=None):
    results = []
    for i, url in enumerate(urls):
        results.append(
            asyncio.create_task(scraper(url, i=i, timeout=60, start=start))
        )
    list_of_links = await asyncio.gather(*results)
    return list_of_links

if __name__ == "__main__":
    set_arsenic_log_level()
    start = time.time()
    urls = [
        'https://www.spoonflower.com/en/shop?on=fabric',
        'https://www.spoonflower.com/en/fabric/6444170-catching-fireflies-by-thestorysmith'
    ]
    name = 'link2.pkl'
    results = asyncio.run(run(urls, start=start))
    print(len(results))
    end = time.time() - start
    print(f'total time is: {end}')

    # loop = asyncio.get_event_loop()
    # result = loop.run_until_complete(scraper(url))
    # df = asyncio.run(run(url))
    # print(df.head())

Overwriting async_scrape_multiple.py


In [5]:
!python async_scrape_multiple.py

Starting ChromeDriver 90.0.4430.212 (e3cd97fc771b893b7fd1879196d1215b622c2bed-refs/branch-heads/4430@{#1429}) on port 35815
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
[1621666293.641][SEVERE]: bind() failed: Cannot assign requested address (99)
ChromeDriver was started successfully.
Starting ChromeDriver 90.0.4430.212 (e3cd97fc771b893b7fd1879196d1215b622c2bed-refs/branch-heads/4430@{#1429}) on port 44285
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
[1621666293.643][SEVERE]: bind() failed: Cannot assign requested address (99)
ChromeDriver was started successfully.
1 took 5.170779228210449 seconds
0 took 6.390865802764893 seconds
2
total time is: 6.457573890686035


In [ ]:
import pandas as pd
name = 'links.pkl'
df = pd.read_pickle(name)
df.head()

,slug,path,scraped
id,,,
10307245,golden-girls-faces-small-teal-by-chica_and_jo,/en/fabric/10307245-golden-girls-faces-small-t...,0
7872207,art-deco-cranes-by-katerhees,/en/fabric/7872207-art-deco-cranes-by-katerhees,0
3730688,william-morris-pimpernel-original-on-black-by-...,/en/fabric/3730688-william-morris-pimpernel-or...,0
5964319,hearts-on-grey-linen-valentines-day-by-littlea...,/en/fabric/5964319-hearts-on-grey-linen-valent...,0
8091225,dark-floral-black-roses-on-black-moody-floral-...,/en/fabric/8091225-dark-floral-black-roses-on-...,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=455dbd66-b743-41fb-bb10-e0d89ff02cc3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>